# Прогнозирование оттока клиентов банка #

**Описание исследования:** Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

**Цель исследования:**  
1. Спрогнозировать, уйдёт клиент из банка в ближайшее время или нет, на основе предоставленных исторических данных о поведении клиентов и расторжении договоров с банком. 
2. Построить модель с предельно большим значением *F1*-меры. (не менее 0,59).
3. Измерить *AUC-ROC*, сравнить её значение с *F1*-мерой.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Метод-Upsampling" data-toc-modified-id="Метод-Upsampling-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Метод Upsampling</a></span></li><li><span><a href="#Метод-Downsampling" data-toc-modified-id="Метод-Downsampling-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Метод Downsampling</a></span></li><li><span><a href="#Использование-параметра-class_weight" data-toc-modified-id="Использование-параметра-class_weight-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Использование параметра class_weight</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li></ul></div>

## Подготовка данных

Импортируем необходимые для работы библиотеки.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

Загрузим данные в переменную и изучим их.

In [3]:
data = pd.read_csv("/datasets/Churn.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/Churn.csv'

In [ ]:
data.info()

In [ ]:
data.head(15)

In [ ]:
data.describe().T

**Описание данных:**

Признаки:
* RowNumber — индекс строки в данных. Тип - int64, подходит значениям столбца.
* CustomerId — уникальный идентификатор клиента. Тип - int64, подходит значениям столбца.
* Surname — фамилия. Тип - object, подходит значениям столбца.
* CreditScore — кредитный рейтинг. Тип - int64, подходит значениям столбца.
* Geography — страна проживания. Тип - object, подходит значениям столбца.
* Gender — пол. Тип - object, возможна замена на более корректный тип.
* Age — возраст. Тип - int64, подходит значениям столбца.
* Tenure — сколько лет человек является клиентом банка. Тип - int64, подходит значениям столбца.
* Balance — баланс на счёте. Тип - float64, возможна замена на более корректный тип.
* NumOfProducts — количество продуктов банка, используемых клиентом. 
* HasCrCard — наличие кредитной карты. Тип - int64, подходит значениям столбца.
* IsActiveMember — активность клиента. Тип - int64, подходит значениям столбца.
* EstimatedSalary — предполагаемая зарплата. Тип - float64, возможна замена на более корректный тип.

Целевой признак:
* Exited — факт ухода клиента. Тип - int64, подходит значениям столбца.

### Вывод

1. Много названий столбцов, несоответствующих требованиям хорошего стиля, их  будет необходимо переименовать.
2. Имеются столбцы с типами данных, которые можно было бы заменить на менее затратные в плане памяти, и возможно некоторые из них будет корректно заменить на идентификаторы в виду малого количества типов объектов.
3. В данных имеются пропуски, их необходимо обработать.
4. Столбцы CustomerId и Surname не влияют на прогнозирование оттока клиентов, поэтому будут удалены.

### Предобработка данных

Для начала исправим названия признаков.

In [ ]:
new_columns = ['row_number', 'customer_id', 'surname', 'credit_score', 'geography',
       'gender', 'age', 'tenure', 'balance', 'num_of_products', 'has_cr_card',
       'is_active_member', 'estimated_salary', 'exited']

In [ ]:
data.columns = new_columns
data.info()

Просмотрим уникальные значения признаков, проверим записи на дубликаты.

In [ ]:
def unique (data):
    for column in data.columns:
        print('Уникальные значения столбца', column)
        print (data[column].sort_values().unique().tolist())
        print ('--------------------------')
    return
unique (data)

In [ ]:
def value_counts (data):
    for column in data.columns:
        print('Значения столбца', column)
        print (data[column].sort_values().value_counts())
        print ('--------------------------')
    return
value_counts (data)

In [ ]:
data[data.duplicated() == True]

In [ ]:
data[data['tenure'].isna() == True]

Теперь необходимо обработать пропуски. Пропуски находятся в столбце tenure - сколько лет является клиентом банка. Этот столбец важен для исследования, но при этом пропусков не очень много - менее 10%. Мне кажется некорректным заменять эти значения медианными, т.к. это окажет влияние на обучение в дальнейшем, других закономерностей по которым можно было бы предположить способ заполнений, мною не найдено. Поэтому избавлюсь от пропусков.

In [ ]:
data = data.fillna(-1)

In [ ]:
data['tenure'] = data['tenure'].astype(object)

Удалим ненужные столбцы.

In [ ]:
data = data.drop (['customer_id', 'surname', 'row_number'], axis = 1)

In [ ]:
data.info()

#### Вывод

Были исправлены названия признаков. Дубликаты и аномалии в значениях не обнаружены. Пропуски в виду их малого количества были удалены. Столбцы, не влияющие на исследование, также были удалены.

## Исследование задачи

Необходимо исследовать баланс классов, обучить модель без учёта дисбаланса. 

Преобразуем категориальные признаки  с помощью метода прямого кодирования.

In [ ]:
data = pd.get_dummies(data, drop_first=True)

Для начала необходимо разделить исходные данные на обучающую, валидационную и тестовую выборки.

In [ ]:
data_train, data_valid_test=train_test_split(data, test_size=0.4, random_state=777)
data_valid, data_test=train_test_split(data_valid_test, test_size=0.5, random_state=777)

Проверим успешное разделение на выборки.

In [ ]:
print (len(data_train))
print (len(data_valid))
print (len(data_test))

Выделим признаки и целевой признак.

In [ ]:
features_train = data_train.drop(['exited'], axis=1)
target_train = data_train['exited']
features_valid = data_valid.drop(['exited'], axis=1)
target_valid = data_valid['exited']
features_test = data_test.drop(['exited'], axis=1)
target_test = data_test['exited']

Масштабируем признаки, т.к. у признаков большой разброс значений.

In [ ]:
scaler = StandardScaler()

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
objects = ['object']
newdf = data.select_dtypes(include=numerics)
newdf_object = data.select_dtypes(include=objects)

In [ ]:
numeric_columns = newdf.columns[:-1]

In [ ]:
numeric_columns

In [ ]:
scaler.fit(features_train[numeric_columns]) 

Преобразуем выборки.

In [ ]:
features_train[numeric_columns] = scaler.transform(features_train[numeric_columns])
features_valid[numeric_columns] = scaler.transform(features_valid[numeric_columns]) 
features_test[numeric_columns] = scaler.transform(features_test[numeric_columns]) 

Посмотрим на распределение ушедших и оставшихся клиентов банка в тренировочном наборе данных.

Распределение далеко от 1:1 - почти 4:1. 

In [ ]:
balance_rate = list(target_train.value_counts(normalize=True))
balance_rate

Обучим модели без учета дисбаланса.

Проверим модель решающего дерева и подберем гиперпараметры.

In [ ]:
best_model = None
best_result = 0
for depth in range(1,20):
    model = DecisionTreeClassifier(random_state=777, max_depth=depth)
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    print("f1 =", end='')
    print(f1_score (target_valid,predict)) 
    result = f1_score(target_valid, predict)
    if result > best_result:
        best_model = model
        best_result = result
        best_max_depth = depth
print ('Лучший результат :', best_result, 'max_depth = ', best_max_depth)

In [ ]:
model = DecisionTreeClassifier(random_state=777, max_depth=7)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1 = f1_score (target_valid,predicted_valid)
f1

Проверим модель случайного леса и подберем гиперпараметры.

In [ ]:
best_model = None
best_result = 0
for est in range(1, 51, 2):
    model = RandomForestClassifier(random_state=777, n_estimators=est) 
    model.fit(features_train, target_train) 
    predict = model.predict(features_valid)
    result = f1_score (target_valid,predict)
    print ("n_estimators =", est, ": ", result)
    if result > best_result:
        best_model = model
        best_result = result
        best_n_estimators = est
print ('Лучший результат :', best_result, 'n_estimators = ', best_n_estimators)

In [ ]:
model = RandomForestClassifier(random_state=777, n_estimators=21)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1 = f1_score (target_valid,predicted_valid)
f1

Проверим модель логистической регрессии.

In [ ]:
model = LogisticRegression(random_state=777, solver='liblinear') 
model.fit(features_train,target_train) 
predicted_valid = model.predict(features_valid)
f1 = f1_score (target_valid,predicted_valid)
f1

Проверим модели на адекватность. Чтобы оценить адекватность модели, создадим константную модель: любому объекту она прогнозирует класс «0».

In [ ]:
target_pred_constant = pd.Series([0] * len(target_valid))

f1 = f1_score (target_valid,predicted_valid)
f1

### Вывод

Наблюдается сильный дисбаланс классов, что плохо сказывается на обучении модели, и из-за этого f1-мера моделей и константной модели почти одинаковы в лучшем случае, и меньше константной модели в худшем. Необходимо использовать методы борьбы с дисбалансом.

Лучшее значение f1-меры показала модель решающего дерева - 0,54.

## Борьба с дисбалансом

Существует три способы борьбы с дисбалансом классов:
1. Upsampling
2. Downsampling
3. Использование параметра class_weight

Попробуем каждый и сравним результаты.

### Метод Upsampling

In [ ]:
def upsample(feature,target,rate):
    feature_one = feature[target == 1]
    target_one = target[target == 1]
    feature_zero = feature[target == 0]
    target_zero = target[target == 0]
    
    features = pd.concat([feature_one]  * round(rate) + [feature_zero] )
    targets = pd.concat([target_one] * round(rate) + [target_zero]  )
    
    features,targets = shuffle(features,targets,random_state = 777)
    
    return features,targets


In [ ]:
features_train_upsample,target_train_upsample = upsample(features_train, target_train, 
                                                         (balance_rate[0]/balance_rate[1]))

Проверим значения f1-меры, которые должен был улучшить метод upsampling. 

Вначале проверим решающее дерево.

In [ ]:
best_model = None
best_result = 0
for depth in range(1,20):
    model = DecisionTreeClassifier(random_state=777, max_depth=depth)
    model.fit(features_train_upsample, target_train_upsample)
    predict = model.predict(features_valid)
    print("f1 =", end='')
    print(f1_score (target_valid,predict)) 
    result = f1_score(target_valid, predict)
    if result > best_result:
        best_model = model
        best_result = result
        best_max_depth = depth
print ('Лучший результат :', best_result, 'max_depth = ', best_max_depth)

In [ ]:
model = DecisionTreeClassifier(random_state=777, max_depth=7)
model.fit(features_train_upsample, target_train_upsample)
predicted_valid = model.predict(features_valid)
f1 = f1_score (target_valid,predicted_valid)
f1

Проверим модель случайного леса.

In [ ]:
best_model = None
best_result = 0
for est in range(1, 151, 2):
    model = RandomForestClassifier(random_state=777, n_estimators=est) 
    model.fit(features_train_upsample, target_train_upsample) 
    predict = model.predict(features_valid)
    result = f1_score (target_valid,predict)
    print ("n_estimators =", est, ": ", result)
    if result > best_result:
        best_model = model
        best_result = result
        best_n_estimators = est
print ('Лучший результат :', best_result, 'n_estimators = ', best_n_estimators)

In [ ]:
model = RandomForestClassifier(random_state=777, n_estimators=101)
model.fit(features_train_upsample, target_train_upsample)
predicted_valid = model.predict(features_valid)
f1 = f1_score (target_valid,predicted_valid)
f1

Проверим модель логистической регрессии.

In [ ]:
model = LogisticRegression(random_state=777, solver='liblinear') 
model.fit(features_train_upsample, target_train_upsample) 
predicted_valid = model.predict(features_valid)
f1 = f1_score (target_valid,predicted_valid)
f1

### Метод Downsampling

In [ ]:
def downsample(feature,target,rate):
    feature_one = feature[target == 1]
    target_one = target[target == 1]
    feature_zero = feature[target == 0]
    target_zero = target[target == 0]
    feature_zero_down = feature_zero.sample(frac = 1/round(rate),random_state = 777)
    target_zero_down =  target_zero.sample(frac = 1/round(rate),random_state = 777)
    features = pd.concat([feature_one]  + [feature_zero_down])
    targets = pd.concat([target_one] + [target_zero_down])
    
    features,targets = shuffle(features,targets,random_state = 777)
    
    return features,targets

In [ ]:
features_train_downsample,target_train_downsample = downsample(features_train,target_train,
                                                                     (balance_rate[0]/balance_rate[1]))

Теперь проверим значения f1-меры у моделей, улучшенных методом downsampling.

Проверим модель решающего дерева.

In [ ]:
best_model = None
best_result = 0
for depth in range(1,20):
    model = DecisionTreeClassifier(random_state=777, max_depth=depth)
    model.fit(features_train_downsample, target_train_downsample)
    predict = model.predict(features_valid)
    print("f1 =", end='')
    print(f1_score (target_valid,predict)) 
    result = f1_score(target_valid, predict)
    if result > best_result:
        best_model = model
        best_result = result
        best_max_depth = depth
print ('Лучший результат :', best_result, 'max_depth = ', best_max_depth)

In [ ]:
model = DecisionTreeClassifier(random_state=777, max_depth=4)
model.fit(features_train_downsample, target_train_downsample)
predicted_valid = model.predict(features_valid)
f1 = f1_score (target_valid,predicted_valid)
f1

Проверим модель случайного леса.

In [ ]:
best_model = None
best_result = 0
for est in range(1, 101, 2):
    model = RandomForestClassifier(random_state=777, n_estimators=est) 
    model.fit(features_train_downsample, target_train_downsample) 
    predict = model.predict(features_valid)
    result = f1_score (target_valid,predict)
    print ("n_estimators =", est, ": ", result)
    if result > best_result:
        best_model = model
        best_result = result
        best_n_estimators = est
print ('Лучший результат :', best_result, 'n_estimators = ', best_n_estimators)

In [ ]:
model = RandomForestClassifier(random_state=777, n_estimators=91)
model.fit(features_train_downsample, target_train_downsample)
predicted_valid = model.predict(features_valid)
f1 = f1_score (target_valid,predicted_valid)
f1

Проверим модель логистической регрессии.

In [ ]:
model = LogisticRegression(random_state=777, solver='liblinear') 
model.fit(features_train_downsample, target_train_downsample) 
predicted_valid = model.predict(features_valid)
f1 = f1_score (target_valid,predicted_valid)
f1

### Использование параметра class_weight

Теперь проверим, как повлияет на значения f1-меры моделей метод взвешивания классов.

Проверим модель решающего дерева.

In [ ]:
best_model = None
best_result = 0
for est in range(1, 51, 2):
    model = DecisionTreeClassifier(random_state=777, max_depth=depth, class_weight='balanced') 
    model.fit(features_train, target_train) 
    predict = model.predict(features_valid)
    print("f1 =", end='')
    print(f1_score (target_valid,predict)) 
    result = f1_score(target_valid, predict)
    if result > best_result:
        best_model = model
        best_result = result
        best_max_depth = depth
print ('Лучший результат :', best_result, 'max_depth = ', best_max_depth)

In [ ]:
model = DecisionTreeClassifier(random_state=777, max_depth=19, class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1 = f1_score (target_valid,predicted_valid)
f1

Проверим модель случайного леса.

In [ ]:
best_model = None
best_result = 0
for est in range(1, 131, 2):
    model = RandomForestClassifier(random_state=777, n_estimators=est, class_weight='balanced') 
    model.fit(features_train, target_train) 
    predict = model.predict(features_valid)
    result = f1_score (target_valid,predict)
    print ("n_estimators =", est, ": ", result)
    if result > best_result:
        best_model = model
        best_result = result
        best_n_estimators = est
print ('Лучший результат :', best_result, 'n_estimators = ', best_n_estimators)

In [ ]:
model = RandomForestClassifier(random_state=777, n_estimators=123, class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1 = f1_score (target_valid,predicted_valid)
f1

Проверим модель логистической регрессии.

In [ ]:
model = LogisticRegression(random_state=777, solver='liblinear', class_weight='balanced') 
model.fit(features_train, target_train) 
predicted_valid = model.predict(features_valid)
f1 = f1_score (target_valid,predicted_valid)
f1

### Вывод

Было улучшено качество моделей (повышено значение f1-меры), с учетом дисбаланса классов. 

**Метод Upsampling:**

* Модель решающего дерева - 0.56
* Модель случайного леса - 0.63
* Модель логистической регрессии - 0.51

**Метод Downsampling:**

* Модель решающего дерева - 0.56
* Модель случайного леса - 0.6
* Модель логистической регрессии - 0.52

**Метод взвешивания классов:**

* Модель решающего дерева - 0.52
* Модель случайного леса - 0.59
* Модель логистической регрессии - 0.51

**До применения методов:**
* Модель решающего дерева - 0.55
* Модель случайного леса - 0.57
* Модель логистической регрессии - 0.35 

В независимости от выбора метода борьбы с дисбалансом или его отсутствия, лучшей моделью является модель случайного леса, худшие показатели f1-меры дает логистическая регрессия - лучшие - случайный лес. 

Лучший результат показала модель случайного леса при применении метода Upsampling - 0.63, так же при применении этого метода оказался лучший результат у модели логистической регрессии. Метод взвешивания классов показал наихудшие результаты среди всех методов.

Значения f1-меры для моделей решающего дерева и случайного леса выросли незначительно - менее 10% (+-0,03 для решающего дерева и +0,02-0,06 для случайного леса). Значение f1-меры значительно выросло у модели логистической регрессии - в 1,45 раза (+0,16-0,17), что позволяет сделать вывод, что дисбаланс классов очень сильно влияет на логистическую регрессию, а другие модели менее подвержены этому.

## Тестирование модели

Проверим лучшую модель на тестовой выборке.

Объеденим тестовую и валидационную выборки для более точного обучения, т.к. валидационная выборка нам больше не нужна.

In [ ]:
features_train_valid = pd.concat([features_train] + [features_valid] )
target_train_valid = pd.concat([target_train] + [target_valid]  )

features_train_valid_upsample,target_train_valid_upsample = upsample(features_train_valid, 
                                                                      target_train_valid, 
                                                                     (balance_rate[0]/balance_rate[1]))

In [ ]:
print (len(features_train_valid))
print (len(target_train_valid))

In [ ]:
model = RandomForestClassifier(random_state=777, n_estimators=101)
model.fit(features_train_valid_upsample, target_train_valid_upsample)
predicted_test = model.predict(features_test)
f1 = f1_score (target_test,predicted_test)
f1

In [ ]:
model = RandomForestClassifier(random_state=777, n_estimators=100)
model.fit(features_train_valid_upsample, target_train_valid_upsample)
predicted_test = model.predict(features_test)
f1 = f1_score (target_test,predicted_test)
f1

Теперь необходимо исследовать метрику *AUC-ROC*.

In [ ]:
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]

auc_roc=roc_auc_score(target_test, probabilities_one_test)

print(auc_roc)

In [ ]:
probabilities_test

In [ ]:
fpr, tpr, thresholds = roc_curve(target_test, probabilities_one_test) 

plt.figure()
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(fpr, tpr)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.ylim([0.0, 1.0])
plt.xlim([0.0, 1.0])
plt.title('ROC-кривая')
plt.show() 

### Вывод

Выбранная модель (модель случайного леса с параметрами random_state=777, n_estimators=100, с классами, сбалансированными методом Upsampling) на тестовом наборе данных показала значение f1-меры равное 0,61. Значение далеко от идеального, но лучше, чем у константной модели (0,35).

Площадь под ROC-кривой составила 0,85. Это значение намного больше, чем у случайной модели (0,5), но далеко от идеала - 1.